<a href="https://colab.research.google.com/github/Dhruv1603/voice_GPT/blob/main/voice_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SpeechRecognition pydub
!apt-get install ffmpeg
!pip install openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


#Google Colab Microphone Input Integration

In [33]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

In [34]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

In [35]:
def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
audio_file_path = record(sec=1)

<IPython.core.display.Javascript object>

#Voice and GPT

In [36]:
# from IPython.display import display, HTML
# import openai
# import os
# from google.colab import output
# from transformers import pipeline
# import ipywidgets as widgets
# from IPython.display import clear_output

In [37]:
from IPython.display import display, HTML
from transformers import pipeline
import ipywidgets as widgets

In [38]:
# Initialize variables
recognized_text = None

In [39]:
# Initialize the text generation pipeline
generator = pipeline('text-generation', model='gpt2')

In [40]:
def handle_recognized_text(text):
    global recognized_text
    recognized_text = text
    output_text.value = f"Recognized Text: {recognized_text}"

In [41]:
def generate_response(prompt):
    response = generator(prompt, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

In [42]:
def generate_response_button_clicked(b):
    global recognized_text
    # Use recognized text if available; otherwise, use text from the input box
    prompt = recognized_text if recognized_text else input_text.value.strip()

    if prompt:
        response = generate_response(prompt)
        ai_response_output.value = f"AI Response: {response}"
    else:
        ai_response_output.value = "No text recognized or provided."

In [43]:
# Create HTML and JavaScript for voice recording
html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>Voice to Text Converter</title>
    <script>
        let recognition;
        let isRecording = false;

        function startRecognition() {
            if (!('webkitSpeechRecognition' in window)) {
                alert("Sorry, your browser doesn't support speech recognition.");
                return;
            }

            recognition = new webkitSpeechRecognition();
            recognition.continuous = false;
            recognition.interimResults = false;

            recognition.onstart = function() {
                document.getElementById("status").innerHTML = "Recording...";
                isRecording = true;
            };

            recognition.onresult = function(event) {
                const transcript = event.results[0][0].transcript;
                document.getElementById("output").innerHTML = transcript;
                isRecording = false;
                document.getElementById("status").innerHTML = "Recording stopped.";
                google.colab.kernel.invokeFunction('handle_recognized_text', [transcript], {}); // Send the text to Python
            };

            recognition.onerror = function(event) {
                alert("Error occurred in recognition: " + event.error);
                isRecording = false;
            };

            recognition.onend = function() {
                if (isRecording) {
                    recognition.start(); // Restart recognition if it ends unexpectedly
                }
            };

            recognition.start();
        }

        function stopRecognition() {
            if (recognition) {
                recognition.stop();
                document.getElementById("status").innerHTML = "Recording stopped.";
            }
        }
    </script>
</head>
<body>
    <h1>Voice to Text Converter</h1>
    <button onclick="startRecognition()" style="padding: 10px; background-color: #4CAF50; color: white; border: none; border-radius: 5px;">Start Recording</button>
    <button onclick="stopRecognition()" style="padding: 10px; background-color: #f44336; color: white; border: none; border-radius: 5px;">Stop Recording</button>
    <p id="status" style="font-weight: bold;"></p>
    <h2>Recognized Text:</h2>
    <p id="output" style="border: 1px solid #ccc; padding: 10px; min-height: 50px; background-color: #f9f9f9;"></p>
</body>
</html>
"""

In [44]:
# Create widgets
output_text = widgets.Textarea(value="", description="Recognized Text:", layout=widgets.Layout(width='100%', height='80px'))
ai_response_output = widgets.Textarea(value="", description="AI Response:", layout=widgets.Layout(width='100%', height='120px'))
input_text = widgets.Textarea(placeholder='Type your text here...', description='Input Text:', layout=widgets.Layout(width='100%', height='80px'))

In [45]:
# Create button for generating response
generate_response_button = widgets.Button(description="Generate Response", button_style='primary', layout=widgets.Layout(width='30%', height='40px'))
generate_response_button.on_click(generate_response_button_clicked)

In [46]:
# Create a box layout for the input fields and buttons
input_section = widgets.VBox([input_text, output_text, generate_response_button, ai_response_output], layout=widgets.Layout(padding='10px', border='1px solid #ddd', border_radius='10px', width='80%', margin='0 auto'))

In [47]:
# Display the HTML for voice recognition
display(HTML(html_code))

# Register the Python function as a callback
output.register_callback('handle_recognized_text', handle_recognized_text)

# Display the input section with the widgets
display(input_section)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
